# Basic Usage Tutorial

This tutorial will walk you through the basic workflow of using Neurodent for rodent EEG analysis.

## Overview

Neurodent provides a streamlined workflow for:
1. Loading EEG recordings from various formats
2. Extracting features from continuous data (Windowed Analysis)
3. Visualizing and analyzing results

Let's get started!

## 1. Installation and Setup

First, ensure you have Neurodent installed:

```bash
pip install neurodent
```

Or with uv:
```bash
uv add neurodent
```

## 2. Import Required Modules

Let's import the necessary modules from Neurodent:

In [ ]:
import sys
from pathlib import Path
import logging

import numpy as np
import matplotlib.pyplot as plt

from neurodent import core, visualization, constants

# Set up logging to see progress
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s", 
    level=logging.INFO, 
    stream=sys.stdout
)
logger = logging.getLogger()

## 3. Configure Temporary Directory (Optional)

Neurodent uses a temporary directory for intermediate files during processing. You can set this to a location with sufficient space:

In [ ]:
# Set temporary directory for intermediate files
# core.set_temp_directory("/path/to/your/temp/dir")

# Or use the default temp directory
print(f"Using temp directory: {core.get_temp_directory()}")

## 4. Load EEG Data

Neurodent supports multiple data formats:
- Binary files (`.bin`)
- SpikeInterface recordings
- MNE objects
- Neuroscope/Neuralynx (`.dat`, `.eeg`)
- Open Ephys (`.continuous`)
- NWB files (`.nwb`)

Let's load data using `LongRecordingOrganizer`:

In [ ]:
# Example: Loading from binary files
# Replace with your actual data path
data_path = Path("/path/to/your/data/folder")
animal_id = "your_animal_id"

# Create LongRecordingOrganizer
# mode options: 'bin', 'si' (SpikeInterface), 'mne', etc.
lro = core.LongRecordingOrganizer(
    base_folder=data_path,
    animal_id=animal_id,
    mode="bin",  # Change based on your data format
)

print(f"Loaded recordings for {animal_id}")
print(f"Number of recordings: {len(lro.recordings)}")

## 5. Create Animal Organizer

The `AnimalOrganizer` wraps the `LongRecordingOrganizer` and provides methods for computing features:

In [ ]:
# Create AnimalOrganizer from LongRecordingOrganizer
ao = visualization.AnimalOrganizer(lro)

print(f"Animal Organizer created for {ao.animal_id}")

## 6. Compute Windowed Analysis Results (WAR)

Now we can compute features from the EEG data. Neurodent extracts features in time windows:

### Available Features:

**Linear Features** (per channel):
- `rms`: RMS amplitude
- `logrms`: Log RMS amplitude
- `ampvar`: Amplitude variance
- `logampvar`: Log amplitude variance
- `psdtot`: Total PSD power
- `logpsdtot`: Log total PSD power
- `psdslope`: PSD slope

**Band Features** (per frequency band: delta, theta, alpha, beta, gamma):
- `psdband`: Band power
- `logpsdband`: Log band power
- `psdbandprop`: Fractional band power
- `logpsdbandprop`: Log fractional band power

**Connectivity Features** (between channels):
- `cohere`: Coherence
- `pcorr`: Pearson correlation

**Spectral Features**:
- `psd`: Full power spectral density

In [ ]:
# Compute windowed analysis with selected features
# You can specify 'all' or list specific features
features = ['rms', 'psdband', 'cohere']

war = ao.compute_windowed_analysis(
    features=features,
    multiprocess_mode='serial'  # Options: 'serial', 'multiprocess', 'dask'
)

print(f"Windowed analysis completed!")
print(f"Features computed: {features}")

## 7. Filter and Clean Data

Neurodent provides filtering methods to remove artifacts and outliers:

In [ ]:
# Apply filtering using method chaining
war_filtered = (
    war
    .filter_logrms_range(z_range=3)  # Remove outliers based on log RMS
    .filter_high_rms(max_rms=500)     # Remove high amplitude artifacts
    .filter_low_rms(min_rms=50)       # Remove low amplitude periods
    .filter_reject_channels_by_session()  # Reject bad channels
)

print("Filtering completed!")

## 8. Aggregate Time Windows

Aggregate data across time windows for analysis:

In [ ]:
# Aggregate time windows
war_filtered.aggregate_time_windows()

print("Time windows aggregated!")

## 9. Basic Visualization

### Single Animal Plots

Use `AnimalPlotter` to visualize data from a single animal:

In [ ]:
# Create plotter for single animal
ap = visualization.AnimalPlotter(war_filtered)

# Plot RMS over time
fig = ap.plot_feature_over_time('rms')
plt.show()

# Plot PSD band powers
fig = ap.plot_psdband()
plt.show()

### Multi-Animal Comparison

To compare across multiple animals, use `ExperimentPlotter`:

In [ ]:
# If you have multiple WARs (from multiple animals)
wars = [war_filtered]  # In practice, you'd load multiple animals

# Create experiment plotter
ep = visualization.ExperimentPlotter(
    wars,
    exclude=['nspike', 'lognspike']  # Features to exclude from plots
)

# Create categorical plot grouped by genotype
g = ep.plot_catplot(
    'rms',
    groupby='genotype',
    kind='box',
    catplot_params={'showfliers': False}
)
plt.show()

# Plot PSD band powers grouped by genotype
g = ep.plot_catplot(
    'psdband',
    groupby='genotype',
    x='genotype',
    hue='band',
    kind='box',
    collapse_channels=True,
    catplot_params={'showfliers': False}
)
plt.show()

## 10. Save Results

You can save your WAR objects for later analysis:

In [ ]:
# Save WAR to disk
output_path = Path("./output") / animal_id
output_path.mkdir(parents=True, exist_ok=True)

war_filtered.to_pickle_and_json(output_path)
print(f"WAR saved to {output_path}")

# Load WAR from disk
war_loaded = visualization.WindowAnalysisResult.load_pickle_and_json(output_path)
print(f"WAR loaded from {output_path}")

## Summary

In this tutorial, you learned how to:

1. Import and configure Neurodent
2. Load EEG data using `LongRecordingOrganizer`
3. Create an `AnimalOrganizer` for feature extraction
4. Compute windowed analysis features (WAR)
5. Filter and clean data
6. Visualize results using `AnimalPlotter` and `ExperimentPlotter`
7. Save and load results

## Next Steps

- **[Data Loading Tutorial](data_loading.ipynb)**: Learn about loading different data formats
- **[Windowed Analysis Tutorial](../tutorials/windowed_analysis.ipynb)**: Deep dive into feature extraction
- **[Visualization Tutorial](../tutorials/visualization.ipynb)**: Advanced plotting techniques
- **[Spike Analysis Tutorial](../tutorials/spike_analysis.ipynb)**: Working with spike-sorted data